In [1]:
#/!\ WARNING /!\
#You need data to train, so launch clearData.ipynb at least once

import pandas as pd 
import numpy as np

import random

from keras.models import *
from keras.layers import *



fileClearedData = "dataCleared.csv"
filePlayers = "players.csv"

#Proportion of the dataset dedicated to the training
trainSize = 9/10
#Minimum accuracy of the NN on the evaluating data
minAccuracyTraining = 0.705

#Predict variables
#Minimum probability of a player winning over an other to bet
minproba = 0.65
#Minimum amount of game of each player to bet
minnbgames = 5
#The earnings are used to link the probability of a player winning and the odds (the odds depending on the betting site)
earnings = 1

Using TensorFlow backend.


In [2]:
#Read the files
clearedData = pd.read_csv(fileClearedData) 
players = pd.read_csv(filePlayers)
dataM = clearedData.values

clearedData.head()

,Winner,Loser,WRank,LRank,Winrate winner,Winrate loser,Nb Game winner,Nb Game loser,Result win,Result loose
0,Bublik A.,Mannarino A.,0.018182,0.023256,0.529412,0.441860,51.0,43.0,1,0
1,Moutet C.,Sandgren T.,0.012346,0.014706,0.535714,0.370370,28.0,27.0,1,0
2,Verdasco F.,Andujar P.,0.020408,0.015625,0.428571,0.291667,14.0,24.0,1,0
3,Bedene A.,Ymer M.,0.017241,0.013158,0.500000,0.421053,36.0,19.0,1,0
4,Chardy J.,Barrere G.,0.018519,0.012048,0.538462,0.333333,26.0,15.0,1,0


In [3]:
#Take the dataset and reverse each game (reverse the winner and the looser) to double the size of the dataset
#It then split the data in training and testing data and finally randomise both
def doubleGame(M,trainSize):
    np.random.shuffle(M)
    M2 =[]
    for match in M:
        reversematch = [match[1],match[0],match[3],match[2],match[5],match[4],match[7],match[6],match[9],match[8]]
        
        M2.append(match)
        M2.append(reversematch)
    Train = np.array(M2[:int(len(M2)*trainSize)])
    Test = np.array(M2[int(len(M2)*trainSize):])
    alldata = np.concatenate((Train, Test), axis=0)
    np.random.shuffle(Train)
    np.random.shuffle(Test)
    
    #TrainX = inputs, TrainY = outputs
    TrainX = np.transpose(np.transpose(Train)[2:8])
    TrainY = np.transpose(np.transpose(Train)[8:])

    TestX = np.transpose(np.transpose(Test)[2:8])
    TestY = np.transpose(np.transpose(Test)[8:])
    return TrainX, TrainY, TestX, TestY, alldata

In [4]:
TrainX, TrainY, TestX, TestY, randM = doubleGame(dataM,trainSize)

print(len(TrainX)," ",len(TestX)," ",len(randM))

3612   402   4014


In [5]:
def createModel():
    #I create the model of the Neural Network
    visible = Input(shape=(6,))
    hidden1 = Dense(250, activation='elu')(visible)
    hidden2 = Dense(100, activation='relu')(hidden1)
    hidden3 = Dropout(0.2) (hidden2)
    hidden4 = Dense(50, activation='relu')(hidden3)
    hidden5 = Dense(20, activation='relu')(hidden4)
    output = Dense(2, activation='softmax')(hidden5)

    model = Model(inputs=visible, outputs=output)

    #I create the neural Network
    model.compile(loss='binary_crossentropy', optimizer='Adagrad', metrics=['accuracy'])
    return model

def trainModel(TrainX,TrainY,model):
    #I fit the model
    model.fit(TrainX, TrainY, epochs=250, batch_size=10,verbose= 2)
    return model

def evaluateModel(TestX,TestY,model):
    #I test on the testing dataset the model i just trained
    score = model.evaluate(TestX, TestY, verbose=0)
    return score

In [6]:
model = createModel()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [7]:
model = trainModel(TrainX,TrainY,model)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/250
 - 0s - loss: 0.6775 - accuracy: 0.6027
Epoch 250/250
 - 0s - loss: 0.5319 - accuracy: 0.7140


In [8]:
score = evaluateModel(TestX,TestY,model)
print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

while score[1] < minAccuracyTraining:
    model = createModel()
    model = trainModel(TrainX,TrainY,model)
    score = evaluateModel(TestX,TestY,model)
    print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')
    

Test loss: 0.4976297047600817 / Test accuracy: 0.7238805890083313


In [9]:
class NotEnoughGame(Exception):
    """One of the player need to play more for the data to be usefull"""
    pass

def prediction(model,p1,p2,minproba,minnbgames,earnings):
    
    rankP1 = float(players.loc[players['Name'] == p1, "Rank"])
    rankP2 = float(players.loc[players['Name'] == p2, "Rank"])
    winrateP1 = float(players.loc[players['Name'] == p1, "Winrate"])
    winrateP2 = float(players.loc[players['Name'] == p2, "Winrate"])
    nbGameP1 = float(players.loc[players['Name'] == p1, "NbGames"])
    nbGameP2 = float(players.loc[players['Name'] == p2, "NbGames"])
    
    if (nbGameP1 < minnbgames) or (nbGameP2 < minnbgames):
        raise NotEnoughGame
    
    dataPredict = np.array([1/rankP1,1/rankP2,winrateP1,winrateP2,nbGameP1,nbGameP2])
    
    #print(dataPredict.shape)
    
    res = model.predict(np.array([dataPredict]))[0]
    
    print(p1,"- rank",rankP1,"- winrate",winrateP1,"- nbGames",nbGameP1)
    print(p2,"- rank",rankP2,"- winrate",winrateP2,"- nbGames",nbGameP2)
    print("Prediction :",res)
    if (res[0]>=minproba):
        print("Bet on",p1,"if odds >",earnings/res[0])
    elif (res[1]>=minproba):
        print("Bet on",p2,"if odds >",earnings/res[1])
    else:
        print("Do not bet")
    return res

In [10]:
#I predict between p1 and p2 which player i should bet (or not bet at all)
#To get the name right, I copy it from the players.csv file

p1 = 'Goffin D.'
p2 = 'Zverev A.'

prediction(model,p1,p2,minproba,minnbgames,earnings)

Goffin D. - rank 13.0 - winrate 0.5142857142857142 - nbGames 35.0
Zverev A. - rank 7.0 - winrate 0.7708333333333334 - nbGames 48.0
Prediction : [0.28577635 0.7142236 ]
Bet on Zverev A. if odds > 1.4001217089593616


array([0.28577635, 0.7142236 ], dtype=float32)